In [2]:
import cv2
import mediapipe as mp
import time
import numpy as np
import math
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

In [16]:
def calculate_balance_time():
    cap = cv2.VideoCapture(0)
    balance_time = 0
    overallTime = None
    start_time = None
    standing = False
    loop = False
    done = False

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark

                # Get coordinates
                leftAnkle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                rightAnkle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                leftHip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                rightHip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                leftKnee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                rightKnee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                leftShoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                rightShoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                # Calculate angle
                angle1 = calculate_angle(rightHip, rightKnee, rightAnkle)
                angle2 = calculate_angle(leftHip, leftKnee, leftAnkle)

                # Calculate balance time

                if (abs(leftAnkle[1] - rightAnkle[1]) > 0.05) and ((leftShoulder[0] > (rightHip[0]) or (rightShoulder[0] < leftHip[0]))):
                    balance_time = 0
                    while not standing:
                        standing = True
                        start_time = time.time()
                        
                balance_time = time.time() - start_time
                overallTime = overallTime + balance_time
                if ((angle2 < 160 and angle1 < 160) or ((leftShoulder[0] < (rightHip[0]) or (rightShoulder[0] > leftHip[0])))):
                    if standing:
                        standing = False
                        balance_time = time.time() - start_time
                        start_time = None
                        # return balance_time
                        # break
                if balance_time > 60:
                    #cv2.putText(image, "Congratulations", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    balance_time = 0
                    start_time = None
                    standing = False
                    done = True
                    
                if ((abs(leftAnkle[1] - rightAnkle[1]) <= 0.05)):
                    if standing:
                        standing = False
                        balance_time = time.time() - start_time
                        start_time = None
                        overallTime = None
                        
                balance_time = time.time() - start_time
                overallTime = overallTime + balance_time
            except:
                pass

            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                      )

            # Display balance time
            if not done:
                cv2.putText(image, "Balance time: " + str(round(overallTime, 2)), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            if done:
                cv2.putText(image, "Congratulations " + str(round(overallTime, 2)), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                break
            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

    return overallTime

calculate_balance_time()

TypeError: type NoneType doesn't define __round__ method

In [14]:
# with break
def calculate_balance_time():
    cap = cv2.VideoCapture(0)
    balance_time = 0
    start_time = None
    standing = False
    loop = False
    done = False

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()

            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Make detection
            results = pose.process(image)

            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark

                # Get coordinates
                leftAnkle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                rightAnkle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                leftHip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                rightHip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                leftKnee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                rightKnee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                leftShoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                rightShoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

                # Calculate angle
                angle1 = calculate_angle(rightHip, rightKnee, rightAnkle)
                angle2 = calculate_angle(leftHip, leftKnee, leftAnkle)

                # Calculate balance time

                if (abs(leftAnkle[1] - rightAnkle[1]) > 0.05) and ((leftShoulder[0] > (rightHip[0]) or (rightShoulder[0] < leftHip[0]))):
                    balance_time = 0
                    while not standing:
                        standing = True
                        start_time = time.time()

                balance_time = time.time() - start_time
                if ((angle2 < 160 and angle1 < 160) or ((leftShoulder[0] < (rightHip[0]) or (rightShoulder[0] > leftHip[0])))):
                    if standing:
                        standing = False
                        balance_time = time.time() - start_time
                        start_time = None
                        return balance_time
                        break
                if balance_time > 60:
                    #cv2.putText(image, "Congratulations", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    balance_time = 0
                    start_time = None
                    standing = False
                    done = True

                if ((abs(leftAnkle[1] - rightAnkle[1]) <= 0.05)):
                    if standing:
                        standing = False
                        balance_time = time.time() - start_time
                        start_time = None

                balance_time = time.time() - start_time
            except:
                pass

            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                      )

            # Display balance time
            if not done:
                cv2.putText(image, "Balance time: " + str(round(balance_time, 2)), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            if done:
                cv2.putText(image, "Congratulations " + str(round(balance_time, 2)), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                break
            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

    return balance_time

calculate_balance_time()

5.060038089752197